# Audio Cluster

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import SpectralClustering

In [2]:
feature_path = "../data/features.csv"
voices_path = "../data/voices.csv"
output_path = "../data/voice_cluster.csv"

In [3]:
features = pd.read_csv(feature_path, index_col="clip_id")
voices = pd.read_csv(voices_path, index_col="clip_id")

In [4]:
min_count = voices['voice_age_group'].value_counts().min()
data = features.join(voices, how='inner')
data = data.groupby('voice_age_group', group_keys=False).apply(
    lambda x: x.sample(n=min_count, random_state=42)
).sample(frac=1, random_state=42)
features = data[features.columns]
voices = data[voices.columns]

/tmp/ipykernel_48075/3937799415.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('voice_age_group', group_keys=False).apply(


In [5]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [6]:
spectral = SpectralClustering(n_clusters=10, affinity='nearest_neighbors', n_neighbors=10, random_state=42)
cluster = spectral.fit_predict(features_scaled)

In [7]:
tsne = TSNE(n_components=3, random_state=42)
embeddings = tsne.fit_transform(features_scaled)

In [8]:
data = features.join(voices, how='left')
data['cluster'] = cluster
data['x'] = embeddings[:, 0]
data['y'] = embeddings[:, 1]
data['z'] = embeddings[:, 2]

In [9]:
data.to_csv(output_path, index_label="clip_id")